# Bank marketing campaigns dataset analysis 
## Opening a Term Deposit
--- 
It is a dataset that describing Portugal bank marketing campaigns results.
Conducted campaigns were based mostly on direct phone calls, offering bank client to place a term deposit. If after all marking affords client had agreed to place deposit - target variable marked 'yes', otherwise 'no'.


|No | Column | Des. |
|:--- | :--- | :--- |
|1| - age|  (numeric)
|2| - job |: type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown") |
|3| - marital| : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
|4| - education| (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
|5| - default |: has credit in default? (categorical: "no","yes","unknown")
|6| - housing| : has housing loan? (categorical: "no","yes","unknown")
|7| - loan | : has personal loan? (categorical: "no","yes","unknown")


# 1. Load Library & Dataset


In [768]:
## EDA Standard Libary

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats as ss

In [769]:
#ML Library

#ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#ML TrainTest Split
from sklearn.model_selection import train_test_split
#ML Report
from sklearn.metrics import  accuracy_score

In [770]:
#Load dataset

df = pd.read_csv("/Users/Dwika/My Projects/Modul 3 Purwadhika/_GROUPCHALLENGE-MODELING/DATA/Train Data.csv")
df.head()

,No,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,1.0,56.0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261.0,1.0,999.0,0.0,nonexistent,no
1,2.0,57.0,services,married,high.school,unknown,no,no,telephone,may,mon,149.0,1.0,999.0,0.0,nonexistent,no
2,3.0,37.0,services,married,high.school,no,yes,no,telephone,may,mon,226.0,1.0,999.0,0.0,nonexistent,no
3,4.0,40.0,admin.,married,basic.6y,no,no,no,telephone,may,mon,151.0,1.0,999.0,0.0,nonexistent,no
4,5.0,56.0,services,married,high.school,no,no,yes,telephone,may,mon,307.0,1.0,999.0,0.0,nonexistent,no


In [771]:
#Construct deep Info on columns & values:

depositData = []
for i in df.columns:
    depositData.append([i, df[i].dtypes,
                      df[i].isna().sum(),
                      round((((df[i].isna().sum())/(len(df)))*100),2), 
                    df[i].nunique(), 
                    df[i].sample(3).values])
pd.DataFrame(depositData, columns = ['dataFeatures', 'dataType', 'null', 'nullPct', 'unique','uniqueSample'])


,dataFeatures,dataType,null,nullPct,unique,uniqueSample
0,No,float64,178,1.7,10310,"[198.0, 734.0, 3720.0]"
1,age,float64,178,1.7,75,"[52.0, 42.0, 37.0]"
2,job,object,178,1.7,12,"[services, admin., retired]"
3,marital,object,178,1.7,4,"[married, married, married]"
4,education,object,178,1.7,8,"[high.school, unknown, university.degree]"
5,default,object,178,1.7,2,"[no, no, no]"
6,housing,object,178,1.7,3,"[unknown, yes, yes]"
7,loan,object,178,1.7,3,"[no, unknown, no]"
8,contact,object,178,1.7,2,"[telephone, telephone, cellular]"
9,month,object,178,1.7,10,"[aug, may, may]"


- Missing values terdapat pada semua kolom data secara seragam, dan presentase nya 1.7 % dari total data, asumsi tiddak mempengaruhi data secra signifikan --> drop all missing values

# 2. Data Cleaning

## 2.1 Drop Missing Values


In [772]:
df.dropna(inplace=True)

In [773]:
#Check missing values setelah cleaning
df.isna().sum()

No             0
age            0
job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
duration       0
campaign       0
pdays          0
previous       0
poutcome       0
y              0
dtype: int64

## 2.2 Check Duplicates

In [774]:
df.duplicated().sum()

0

Tidak ada duplikat data

## 2.3 Replace unknowns

In [775]:

#Check "unknown" in several columns
def CheckUnknown(col):
    print(col, ':', len(df[df[col] == 'unknown'])/len(df)* 100)

for item in df.columns:
    CheckUnknown(item)    

No : 0.0
age : 0.0
job : 1.0766246362754608
marital : 0.20368574199806014
education : 5.237633365664403
default : 22.230843840931136
housing : 2.7352085354025215
loan : 2.7352085354025215
contact : 0.0
month : 0.0
day_of_week : 0.0
duration : 0.0
campaign : 0.0
pdays : 0.0
previous : 0.0
poutcome : 0.0
y : 0.0


In [776]:
colWithUnknown = ['job', 'marital', 'education', 'housing', 'loan']

## 2.4 Replace Unknown Values  pada kolom default berdasarkan 'housing' dan 'loan'

In [777]:
df[['default', 'housing', 'loan' ]]

,default,housing,loan
0,no,no,no
1,unknown,no,no
2,no,yes,no
3,no,no,no
4,no,no,yes
...,...,...,...
10483,unknown,no,no
10484,no,yes,no
10485,no,yes,no
10486,no,no,no


In [778]:
#Check the correlation between columns of default vs housing and loan
print(df[(df['housing'] == 'yes') & (df['loan'] == 'yes')]['default'].value_counts())
print(df[(df['housing'] == 'no') & (df['loan'] == 'yes')]['default'].value_counts())
print(df[(df['housing'] == 'yes') & (df['loan'] == 'no')]['default'].value_counts())
print(df[(df['housing'] == 'no') & (df['loan'] == 'no')]['default'].value_counts())


default
no         673
unknown    200
Name: count, dtype: int64
default
no         522
unknown    135
Name: count, dtype: int64
default
no         3321
unknown     899
Name: count, dtype: int64
default
no         3298
unknown     980
Name: count, dtype: int64


**VALUE REPLACEMENT FOR DEFAULT BY HOUSING AND LOAN**

| Housing | Loan | Replace as | 
| :--- | :--- | :--- | 
| no | no | no |
| yes | no | yes |
| no | yes | yes |
| yes | no | no |

In [779]:
#converting default column based on condition of housing and loan
def convertDefault(col):
    if col['housing'] == 'yes' and col['loan'] == 'yes':
        return 'no'
    elif col['housing'] == 'no' and col['loan'] == 'yes':
        return 'no'
    elif col['housing'] == 'yes' and col['loan'] == 'no':
        return 'no'
    elif col['housing'] == 'no' and col['loan'] == 'no':
        return 'yes'
    else:
        return col['default']
    
df['default'] = df.apply(convertDefault, axis=1)
df['default'].value_counts()

default
no         5954
yes        4278
unknown      78
Name: count, dtype: int64

In [847]:
for item in df.columns:
    CheckUnknown(item)

age : 0.0
job : 1.0766246362754608
marital : 0.20368574199806014
education : 5.237633365664403
default : 0.7565470417070805
housing : 2.7352085354025215
loan : 2.7352085354025215
campaign : 0.0
previous : 0.0
y : 0.0


# 3. Feature Engineering

## 3.1 Target Column

In [781]:
# Convert y column to 0 & 1[ 

df['y'].replace({'yes': 1, 'no': 0}, inplace=True)

In [782]:
df['y'].value_counts()

y
0    5810
1    4500
Name: count, dtype: int64

## 3.2 Pilih Kolom data

In [783]:
#Kolom existing
df.columns

Index(['No', 'age', 'job', 'marital', 'education', 'default', 'housing',
       'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'y'],
      dtype='object')

In [784]:
#Construct deep Info on columns & values:

depositData = []
for i in df.columns:
    depositData.append([i, df[i].dtypes,
                      df[i].isna().sum(),
                      round((((df[i].isna().sum())/(len(df)))*100),2), 
                    df[i].nunique(), 
                    df[i].sample(3).values])
pd.DataFrame(depositData, columns = ['dataFeatures', 'dataType', 'null', 'nullPct', 'unique','uniqueSample'])


,dataFeatures,dataType,null,nullPct,unique,uniqueSample
0,No,float64,0,0.0,10310,"[25854.0, 38558.0, 35949.0]"
1,age,float64,0,0.0,75,"[54.0, 56.0, 59.0]"
2,job,object,0,0.0,12,"[admin., admin., housemaid]"
3,marital,object,0,0.0,4,"[married, married, single]"
4,education,object,0,0.0,8,"[basic.6y, basic.4y, university.degree]"
5,default,object,0,0.0,3,"[no, no, no]"
6,housing,object,0,0.0,3,"[yes, no, no]"
7,loan,object,0,0.0,3,"[no, no, no]"
8,contact,object,0,0.0,2,"[telephone, cellular, cellular]"
9,month,object,0,0.0,10,"[may, sep, may]"


## 3.3 Convert Categorical Columns

### 3.3.1 Convert Education Columns

In [785]:
education_mapping = {
    'university.degree': 'High Education',
    'basic.9y': 'High Education',
    'professional.course': 'High Education',
    'basic.4y': 'High Education',
    'basic.6y': 'High Education',
    'illiterate': 'Non High Education',
    'high.school': 'Non High Education'
}

df['education'] = df['education'].replace(education_mapping)
df['education'].value_counts()

education
High Education        7388
Non High Education    2382
unknown                540
Name: count, dtype: int64

### 3.3.2 Convert Job Columns

In [786]:
#Map Convertion
job_mapping = {
    'technician': 'blue-collar',
    'services': 'blue-collar',
    'housemaid': 'blue-collar',
    'admin.': 'white-collar',
    'management': 'white-collar',
    'entrepreneur': 'self-employed',
    'student': 'unemployed',
}

# Replace the job categories in the dataset
df['job'] = df['job'].replace(job_mapping)
df['job'].value_counts()

job
blue-collar      5104
white-collar     3265
self-employed     665
unemployed        585
retired           580
unknown           111
Name: count, dtype: int64

### 3.3.3 Convert Marital Column

In [787]:
df['marital'] = df['marital'].replace({'divorced': 'single'})
df['marital'].value_counts()

marital
married    6402
single     3887
unknown      21
Name: count, dtype: int64

# 3.4 Columns Used

In [788]:
df.head()

,No,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,1.0,56.0,blue-collar,married,High Education,yes,no,no,telephone,may,mon,261.0,1.0,999.0,0.0,nonexistent,0
1,2.0,57.0,blue-collar,married,Non High Education,yes,no,no,telephone,may,mon,149.0,1.0,999.0,0.0,nonexistent,0
2,3.0,37.0,blue-collar,married,Non High Education,no,yes,no,telephone,may,mon,226.0,1.0,999.0,0.0,nonexistent,0
3,4.0,40.0,white-collar,married,High Education,yes,no,no,telephone,may,mon,151.0,1.0,999.0,0.0,nonexistent,0
4,5.0,56.0,blue-collar,married,Non High Education,no,no,yes,telephone,may,mon,307.0,1.0,999.0,0.0,nonexistent,0


In [789]:
df = df.drop(['No','contact','month','day_of_week','poutcome', 'duration', 'pdays'],axis=1)

In [790]:
df

,age,job,marital,education,default,housing,loan,campaign,previous,y
0,56.0,blue-collar,married,High Education,yes,no,no,1.0,0.0,0
1,57.0,blue-collar,married,Non High Education,yes,no,no,1.0,0.0,0
2,37.0,blue-collar,married,Non High Education,no,yes,no,1.0,0.0,0
3,40.0,white-collar,married,High Education,yes,no,no,1.0,0.0,0
4,56.0,blue-collar,married,Non High Education,no,no,yes,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
10483,57.0,retired,married,Non High Education,yes,no,no,1.0,0.0,0
10484,40.0,blue-collar,single,Non High Education,no,yes,no,1.0,0.0,0
10485,28.0,blue-collar,single,Non High Education,no,yes,no,1.0,0.0,0
10486,34.0,blue-collar,married,High Education,yes,no,no,1.0,0.0,0


# 4. OHE ( One Hot Encoding )

In [791]:
df.columns


Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'campaign', 'previous', 'y'],
      dtype='object')

In [792]:
#Convert sex & alone col to dummy var, save to a variable

df_ML =  pd.get_dummies(df, columns=['job', 'marital', 'education', 'default', 'housing',
       'loan'], dtype=int, drop_first=True)
df_ML

,age,campaign,previous,y,job_retired,job_self-employed,job_unemployed,job_unknown,job_white-collar,marital_single,marital_unknown,education_Non High Education,education_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes
0,56.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,57.0,1.0,0.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,37.0,1.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,40.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,56.0,1.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10483,57.0,1.0,0.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
10484,40.0,1.0,0.0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
10485,28.0,1.0,0.0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
10486,34.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# 3 Train Test Split


## 3.1 Train Data

In [825]:
#Train test split
x = df_ML.drop('y', axis=1)  # All cols except default as ind var (x)
y = df_ML['y']               # default col as target (y)

xtrain, xtest, ytrain, ytest = train_test_split(
    x,
    y,
    test_size= 0.2,    # Jumlah size untuk test data, size training akan ngikut,
    random_state=20,   # Random seed
    stratify=y         # Proporsi antar train & test dari y yg diambil disamakan
)

## 3.2 Load Test Data


In [826]:
#Load test data
test = pd.read_csv("/Users/Dwika/My Projects/Modul 3 Purwadhika/_GROUPCHALLENGE-MODELING/DATA/Test Data.csv")

In [827]:
test.head()

,No,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome
0,40805,35,technician,married,professional.course,no,yes,no,cellular,sep,thu,1,999,2,failure
1,40807,35,technician,married,professional.course,no,yes,no,cellular,sep,thu,1,999,2,failure
2,40808,54,admin.,divorced,university.degree,no,yes,no,cellular,sep,thu,1,999,2,failure
3,40809,35,technician,married,professional.course,no,yes,no,cellular,sep,thu,2,999,0,nonexistent
4,40810,33,admin.,married,university.degree,no,yes,no,telephone,sep,thu,2,999,0,nonexistent


In [828]:
#Convert test data with same method as train data
#Job Convert

test['job'] = test['job'].replace(job_mapping)
test['job'].value_counts()

job
white-collar     126
blue-collar      114
unemployed        58
retired           46
self-employed      9
unknown            2
Name: count, dtype: int64

In [829]:
#Convert education
test['education'] = test['education'].replace(education_mapping)
test['education'].value_counts()

education
High Education        243
Non High Education     92
unknown                20
Name: count, dtype: int64

In [830]:
#Convert marital
test['marital'] = test['marital'].replace({'divorced': 'single'})
test['marital'].value_counts()

marital
single     188
married    166
unknown      1
Name: count, dtype: int64

In [831]:
test = test.drop(['No', 'contact','month','day_of_week','poutcome', 'pdays'],axis=1)

In [832]:
test.head()

,age,job,marital,education,default,housing,loan,campaign,previous
0,35,blue-collar,married,High Education,no,yes,no,1,2
1,35,blue-collar,married,High Education,no,yes,no,1,2
2,54,white-collar,single,High Education,no,yes,no,1,2
3,35,blue-collar,married,High Education,no,yes,no,2,0
4,33,white-collar,married,High Education,no,yes,no,2,0


In [833]:
#Convert Default
test['default'] = test.apply(convertDefault, axis=1)
test['default'].value_counts()

default
no         234
yes        120
unknown      1
Name: count, dtype: int64

In [834]:
#Convert col to dummy var, save to a variable
test =  pd.get_dummies(test, columns=['job', 'marital', 'education', 'default', 'housing','loan'], dtype=int, drop_first=True)
test

,age,campaign,previous,job_retired,job_self-employed,job_unemployed,job_unknown,job_white-collar,marital_single,marital_unknown,education_Non High Education,education_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes
0,35,1,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,35,1,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,54,1,2,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0
3,35,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,33,2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,73,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
351,46,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
352,56,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
353,44,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# 4. Model


# 4.1 Internal Test Data

### 4.1.3 Logistic Regression

In [835]:
#Build model

logreg = LogisticRegression()
logreg.fit(xtrain,ytrain) #apply model to training data (xtrain,ytrain)

/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [836]:
#Print report
pred_logreg = logreg.predict(xtest)
print(f'Logistic Regression Accuracy Score: {round(accuracy_score(ytest,pred_logreg)*100,2)} %')

Logistic Regression Accuracy Score: 72.7 %


In [837]:
logreg.predict(xtest)

array([1, 0, 0, ..., 0, 0, 0])

### 4.1.2 KNN

In [838]:
#Build model

knn = KNeighborsClassifier()
knn.fit(xtrain,ytrain) #apply model to training data (xtrain,ytrain)

KNeighborsClassifier()

In [839]:
#Print report
pred_knn = knn.predict(xtest.values)
print(f'KNN Accuracy Score: {round(accuracy_score(ytest,pred_knn)*100,2)} %')

KNN Accuracy Score: 67.17 %


/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [808]:
accuracies = []
kvalues = []
k_score_dictionary = {}

for k in range(1, 10, 2):
    knn_multi_k = KNeighborsClassifier(n_neighbors=k)
    knn_multi_k.fit(xtrain, ytrain)
    # Print report
    pred_knn = knn_multi_k.predict(xtest.values)
    acc = accuracy_score(ytest, pred_knn)
    kvalues.append(k)  # Append the k value to kvalues
    accuracies.append(acc)  # Append the accuracy to accuracies
    k_score_dictionary[k] = acc  # Store accuracy in the dictionary

# Create a DataFrame to store k-values and their corresponding accuracies
df_score_result = pd.DataFrame({'k': kvalues, 'accuracy': accuracies})

# Print the DataFrame
df_score_result

/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/Dwika/My Projects/.venv/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  

,k,accuracy
0,1,0.636760
1,3,0.650824
2,5,0.671678
3,7,0.687197
4,9,0.688167


In [809]:
df_score_result.sort_values(by='accuracy', ascending=False).head(5)

,k,accuracy
4,9,0.688167
3,7,0.687197
2,5,0.671678
1,3,0.650824
0,1,0.636760


## 3.1.3 Decision Tree

In [840]:
#Modeling with Decision Tree, varying max_depth parameter 
#Create looping

md = np.arange(1, 10,1)
crit = ['gini', 'entropy']
acc_score = []
krit = []

for i in md:
    for j in crit:
        dtc = DecisionTreeClassifier(max_depth=i, criterion=j, random_state=2020)
        dtc.fit(xtrain, ytrain)
        ypred = dtc.predict(xtest)
        acc_score.append(accuracy_score(ytest, ypred))
        krit.append((i,j))

In [812]:
# param_grid = {
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [10],
#     'min_samples_split': [3],
#     'min_samples_leaf': [4],
#     'max_features': ['sqrt', 'log2', None],
#     'max_leaf_nodes': [None, 10, 20, 30],
#     'min_impurity_decrease': [0.0, 0.1, 0.2],
#     'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
#     'splitter': ['best', 'random'],
#     'class_weight': [None, 'balanced'],
#     'ccp_alpha': [0.0, 0.1, 0.2]
# }



In [813]:
# grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(xtrain, ytrain)


In [814]:
# best_params = grid_search.best_params_
# best_dt_classifier = grid_search.best_estimator_


In [842]:
#Create dataframe of result

df_result = pd.DataFrame({'criterion':krit, 'accuracy_score':acc_score})

In [844]:
df_result.sort_values(by='accuracy_score', ascending=False).head(5)

,criterion,accuracy_score
10,"(6, gini)",0.736178
11,"(6, entropy)",0.736178
9,"(5, entropy)",0.730359
6,"(4, gini)",0.730359
7,"(4, entropy)",0.730359


# 4.2 Submission Test Data

In [821]:
logreg.predict(test)

array([1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,